In [1]:
import sklearn


In [2]:
# from sklearn.metrics._scorer import _PredictScorer


In [3]:
import pandas as pd
import csv
import os
import numpy as np
from pycaret.survival_analysis.pycaret_wrapper.oop import SurvivalExperiment
from pycaret.survival_analysis.pycaret_wrapper.oop import *
from pycaret.regression.oop import RegressionExperiment
import lifelines
exp1 = SurvivalExperiment()
# data_dir = '..\\csv\\'
# List the files in the current directory
files = os.listdir('./csv')
# Get the absolute path of the current directory
current_dir = os.getcwd()
# Create a DataFrame with the file names and their absolute paths
file_paths = pd.DataFrame({
    'file_name': files,
    'absolute_path': [os.path.join(current_dir, 'csv', file) for file in files]
}) 
df = pd.read_csv(file_paths['absolute_path'][0], index_col=0)
# df = pd.read_csv("../csv/SA/NKI_cleaned.csv", sep=',', encoding='utf-8')

features_to_drop = df.columns[16:]
df = df.drop(features_to_drop, axis=1)
for cols in df.columns:
    print(cols)


ID
age
eventdeath
survival
timerecurrence
chemo
hormonal
amputation
histtype
diam
posnodes
grade
angioinv
lymphinfil
barcode
esr1


In [ ]:
print("DataFrame columns before setup:", df.columns.tolist())
print("Target argument:", ['eventdeath', 'timerecurrence'])
print("First few rows:\n", df.head())

# Create a new experiment for debugging
exp1 = SurvivalExperiment()

# Try to manually inspect what happens during setup
print("\nBefore setup:")
print("exp1.train exists:", hasattr(exp1, 'train'))
print("exp1.X_train exists:", hasattr(exp1, 'X_train'))

# Run partial setup but catch the error
try:
    exp1.setup(data=deepcopy(df),
               target=['eventdeath', 'timerecurrence'],
               ignore_features=['Patient','ID', 'barcode'],
               keep_features=['timerecurrence'],
               fold=10,
               numeric_imputation='median',
               categorical_imputation='mode',
               imputation_type='simple',
               preprocess=True,
               normalize=False,
               n_jobs=1,
               verbose=6)
except Exception as e:
    print(f"Error during setup: {e}")
    print("Debug info:")
    print("exp1.train exists:", hasattr(exp1, 'train'))
    print("exp1.X_train exists:", hasattr(exp1, 'X_train'))
    print("exp1.dataset exists:", hasattr(exp1, 'dataset'))
    print("exp1.pipeline exists:", hasattr(exp1, 'pipeline'))
    print("exp1.idx exists:", hasattr(exp1, 'idx'))
    
    if hasattr(exp1, 'dataset'):
        print("exp1.dataset shape:", exp1.dataset.shape)
        print("exp1.dataset columns:", exp1.dataset.columns.tolist())
        print("exp1.dataset target_param:", exp1.target_param)
        
    if hasattr(exp1, 'idx'):
        print("exp1.idx:", exp1.idx)
        print("exp1.idx type:", type(exp1.idx))
        
    if hasattr(exp1, 'X_train'):
        print("exp1.X_train:", exp1.X_train)
        print("exp1.X_train type:", type(exp1.X_train))
        
    if hasattr(exp1, 'y_train'):
        print("exp1.y_train:", exp1.y_train)
        print("exp1.y_train type:", type(exp1.y_train))
        
    if hasattr(exp1, 'pipeline'):
        print("exp1.pipeline:", exp1.pipeline)
        print("exp1.pipeline steps:", exp1.pipeline.steps)
        
    # Try to call the transform method manually
    if hasattr(exp1, 'X_train') and hasattr(exp1, 'y_train') and hasattr(exp1, 'pipeline'):
        try:
            result = exp1.pipeline.transform(X=exp1.X_train, y=exp1.y_train, filter_train_only=False)
            print("Transform result:", result)
            print("Transform result type:", type(result))
        except Exception as te:
            print(f"Transform error: {te}")
    raise
else:
    print("Setup completed successfully.")
    print("exp1.train exists:", hasattr(exp1, 'train'))
    print("exp1.X_train exists:", hasattr(exp1, 'X_train'))
    print("exp1.dataset exists:", hasattr(exp1, 'dataset'))
    print("exp1.pipeline exists:", hasattr(exp1, 'pipeline'))
    print("exp1.idx exists:", hasattr(exp1, 'idx'))
    
    if hasattr(exp1, 'dataset'):
        print("exp1.dataset shape:", exp1.dataset.shape)
        print("exp1.dataset columns:", exp1.dataset.columns.tolist())
        print("exp1.dataset target_param:", exp1.target_param)
    
    if hasattr(exp1, 'idx'):
        print("exp1.idx:", exp1.idx)
        print("exp1.idx type:", type(exp1.idx))
    
    if hasattr(exp1, 'X_train'):
        print("exp1.X_train shape:", exp1.X_train.shape)
        print("exp1.X_train type:", type(exp1.X_train))
    
    if hasattr(exp1, 'y_train'):
        print("exp1.y_train shape:", exp1.y_train.shape)
        print("exp1.y_train type:", type(exp1.y_train))
    
    if hasattr(exp1, 'pipeline'):
        print("exp1.pipeline steps:", exp1.pipeline.steps)
        print("exp1.pipeline type:", type(exp1.pipeline))
        print("Pipeline steps:")
        for step_name, step in exp1.pipeline.steps:
            print(f"  {step_name}: {type(step)}")   

DataFrame columns before setup: ['ID', 'age', 'eventdeath', 'survival', 'timerecurrence', 'chemo', 'hormonal', 'amputation', 'histtype', 'diam', 'posnodes', 'grade', 'angioinv', 'lymphinfil', 'barcode', 'esr1']
Target argument: ['eventdeath', 'timerecurrence']
First few rows:
          ID  age  eventdeath   survival  timerecurrence  chemo  hormonal  \
Patient                                                                    
s122     18   43           0  14.817248       14.817248      0         0   
s123     19   48           0  14.261465       14.261465      0         0   
s124     20   38           0   6.644764        6.644764      0         0   
s125     21   50           0   7.748118        7.748118      0         1   
s126     22   38           0   6.436687        6.318960      0         0   

         amputation  histtype  diam  posnodes  grade  angioinv  lymphinfil  \
Patient                                                                      
s122              1         1    

KeyError: 'time'

In [ ]:
from sksurv.metrics import *
import sksurv.metrics as s_metrics
from sklearn.metrics import *
exp1.add_metric('logloss', 'Log Loss', log_loss, greater_is_better = False)

# exp1.add_metric('ci', 'C-Index', s_metrics.concordance_index_censored, greater_is_better = False)

def ci_score(y_true, y_pred):
    return s_metrics.concordance_index_censored(y_true, y_pred)

# exp1.add_metric()
print("===========================================================")
print("available models:")
print("===========================================================")
print(exp1.models())
print(exp1.get_metrics())

print("===========================================================")
print("compare models:")
print("===========================================================")
exp1.compare_models(sort='ci', errors='raise')
print("===========================================================")
print("training model:")
print("===========================================================")
model = exp1.create_model('coxnet',  cross_validation=True, return_train_score=True)
# best_model = exp1.compare_models()
print("===========================================================")
print("prediction:")
print("===========================================================")
exp1.predict_model(model)
print('===========================================================')
print('===========================================================')


print("===========================================================")
print("available models:")
print("===========================================================")
print(exp1.models())
print(exp1.get_metrics())

print("===========================================================")
print("compare models:")
print("===========================================================")
models_ = exp1.compare_models(sort='ci-ipcw', errors='raise', n_select=3)
print("===========================================================")
print("training model:")
print("===========================================================")
model1 = exp1.create_model('coxnet', cross_validation=True, return_train_score=True)



print("===========================================================")
print("Plotting:")
print("===========================================================")

exp1.plot_model(model1, plot='plot_coefficients')
exp1.plot_model(model1, plot='plot_cindex')
exp1.plot_model(model1, 'plot_survival_curve')
exp1.plot_model(model1, plot='plot_cauc')
exp1.plot_model(model1, plot='plot_nzcoefs')
exp1.plot_model(model1, 'plot_coefficients')

print('===========================================================')
print('===========================================================')
print('FIN')


ValueError: setup() needs to be ran first.